<a href="https://colab.research.google.com/github/ga5am3/DRL-for-microgrid-energy-management/blob/master/SARSA_microgrid_hyperparameter_sweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/wilsonhu7777/DRL-for-microgrid-energy-management.git

Cloning into 'DRL-for-microgrid-energy-management'...
remote: Enumerating objects: 1402, done.
remote: Counting objects: 100% (1293/1293), done.
remote: Compressing objects: 100% (563/563), done.
remote: Total 1402 (delta 732), reused 1252 (delta 717), pack-reused 109
Receiving objects: 100% (1402/1402), 12.60 MiB | 10.26 MiB/s, done.
Resolving deltas: 100% (775/775), done.


In [ ]:
!cd ..
!cd /; ls; cd /content; ls

bin	 dev   lib32  NGC-DL-CONTAINER-LICENSE	root  sys    var
boot	 etc   lib64  opt			run   tmp
content  home  media  proc			sbin  tools
datalab  lib   mnt    python-apt		srv   usr
DRL-for-microgrid-energy-management  sample_data


In [ ]:
!cd DRL-for-microgrid-energy-management/; mv * /content

In [ ]:
%pip install wandb -q
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8 MB 8.9 MB/s 
     |████████████████████████████████| 181 kB 70.0 MB/s 
     |████████████████████████████████| 158 kB 72.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 157 kB 58.5 MB/s 
     |████████████████████████████████| 157 kB 46.1 MB/s 
     |████████████████████████████████| 157 kB 48.6 MB/s 
     |████████████████████████████████| 157 kB 50.0 MB/s 
     |████████████████████████████████| 157 kB 60.5 MB/s 
     |████████████████████████████████| 157 kB 70.6 MB/s 
     |████████████████████████████████| 157 kB 50.5 MB/s 
     |████████████████████████████████| 156 kB 46.7 MB/s 


In [ ]:
import random, numpy, math, gym
import time
NAME = "SARSA-{}".format(int(time.time()))

# -------------------- BRAIN ---------------------------
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.models import *
from keras.layers import *
from keras import backend as K

import time
NAME = "SARSA-{}".format(int(time.time()))

In [ ]:
DAY0 = 50
DAYN = 60
REWARDS = {}

In [ ]:
for i in range(0,DAYN,1):
    REWARDS[i]=[]

In [ ]:
class Memory:  # stored as ( s, a, r, s_ )
    samples = []

    def __init__(self, capacity):
        self.capacity = capacity

    def add(self, sample):
        self.samples.append(sample)
        if len(self.samples) > self.capacity:
            self.samples.pop(0)

    def extra(self, next_a):
        if len(self.samples) > 1:
            self.samples[-2][-1] = next_a

    def sample(self, n):
        n = min(n, len(self.samples))
        return random.sample(self.samples, n)


In [ ]:
MAX_EPSILON = 0.5
MIN_EPSILON = 0.004

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam

class Brain:
    def __init__(self, stateCnt, actionCnt,optimizer):
        self.stateCnt = stateCnt
        self.actionCnt = actionCnt
        self.opt = optimizer

        self.model = self._createModel()
        #self.tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
        # self.model.load_weights("cartpole-basic.h5")

    def _createModel(self):
        l_input = Input(batch_shape=(None, self.stateCnt))
        l_input1 = Lambda(lambda x: x[:, 0:self.stateCnt - 7])(l_input)
        l_input2 = Lambda(lambda x: x[:, -7:])(l_input)
        l_input1 = Reshape((DEFAULT_NUM_TCLS, 1))(l_input1)
        l_Pool = AveragePooling1D(pool_size=self.stateCnt - 7)(l_input1)
        l_Pool = Reshape([1])(l_Pool)
        l_dense = Concatenate()([l_Pool, l_input2])
        l_dense = Dense(100, activation='relu')(l_dense)
        l_dense = Dropout(0.3)(l_dense)
        out_value = Dense(80, activation='linear')(l_dense)
        # model = Model(inputs=l_input, outputs=[out_tcl_actions,out_price_actions,out_deficiency_actions,out_excess_actions, out_value])
        model = Model(inputs=l_input, outputs=out_value)
        model.make_predict_function()
        opt = RMSprop(lr=0.00025)

        if self.opt=='rmsprop':
          optimizer = RMSprop(lr=(0.00025), decay=1e-5)
        elif self.opt=='adam':
          optimizer = Adam(lr=(0.00025), beta_1=0.9, beta_2=0.999, clipnorm=1.0)

        model.compile(loss='mse', optimizer=self.opt)
        return model


    def train(self, x, y, epoch=1, verbose=0):
        self.model.fit(x, y, batch_size=100, epochs=epoch, verbose=verbose, callbacks=[WandbCallback()])

    def predict(self, s):
        return self.model.predict(s)

    def predictOne(self, s):
        return self.predict(s.reshape(1, self.stateCnt)).flatten()

In [ ]:
GAMMA = 1.
class Agent:
    steps = 0
    epsilon = MAX_EPSILON

    def __init__(self, stateCnt, actionCnt, memory_capacity, batch_size, epsilon, opt):
        self.stateCnt = stateCnt
        self.actionCnt = actionCnt
        self.memory_capacity = memory_capacity
        self.epsilon = epsilon
        self.batch_size = batch_size

        self.brain = Brain(stateCnt, actionCnt, opt)
        self.memory = Memory(self.memory_capacity)

    def act(self, s, deter):
        if deter==True:
            return numpy.argmax(self.brain.predictOne(s))
        if random.random() < self.epsilon:
            return random.randint(0, self.actionCnt - 1)

        return numpy.argmax(self.brain.predictOne(s))

    def observe(self, sample):  # in (s, a, r, s_) format
        self.memory.add(sample)
        self.memory.extra(sample[1])
        # slowly decrease Epsilon based on our eperience
        self.steps += 1
        self.epsilon = max(MAX_EPSILON -self.epsilon * self.steps, MIN_EPSILON)

    def replay(self):
        if len(self.memory.samples)<2:
            return
        batch = self.memory.sample(self.batch_size)
        batchLen = len(batch)

        no_state = numpy.zeros(self.stateCnt)

        states = numpy.array([o[0] for o in batch])
        states_ = numpy.array([(no_state if o[3] is None else o[3]) for o in batch])

        p = self.brain.predict(states)
        p_ = self.brain.predict(states_)

        x = numpy.zeros((batchLen, self.stateCnt))
        y = numpy.zeros((batchLen, self.actionCnt))

        for i in range(batchLen):
            o = batch[i]
            s = o[0]
            a = o[1]
            r = o[2]
            s_ = o[3]
            a_ = o[4]

            t = p[i]
            if s_ is None:
                t[a] = r
            else:
                t[a] = r + GAMMA * p_[i][a_]

            x[i] = s
            y[i] = t

        self.brain.train(x, y)

In [ ]:
from tcl_env_dqn_1 import *

class Environment:
    def __init__(self, render = False, **kwargs):
        self.env = MicroGridEnv(**kwargs)
        self.render=render
        self.run_count = 0

    def run(self, agent, day=None):
        s = self.env.reset(day=day)
        R = 0
        while True:

            a = agent.act(s, deter=self.render)

            s_, r, done, info = self.env.step(a)
            if self.render: self.env.render('SARSA')

            if done:  # terminal state
                s_ = None

            agent.observe([s, a, r, s_,None])


            s = s_
            R += r
            #wandb.log({"r":r})

            if done:

                agent.replay()
                break

        REWARDS[self.env.day].append(R)
        wandb.log({"Reward":R,"run_count":self.run_count, "Day": self.env.day})
        self.run_count += 1
        print("Day ", self.env.day)
        print("R= ", R)

In [ ]:
sweep_config = {
    'method':'bayes',
    'metric': {
      'name': 'Reward',
      'goal': 'maximize'   
    },
    'parameters': {
        'batch_size_replay': {
            'values': [250,275, 300,325]
            },
        'memory_capacity': {'values': [475,500, 525]},
        'optimizer': {'values': ['rmsprop']},
        'epsilon': {'values': [1.0e-6, 1.0e-5, 5e-6]}
    }
}

In [ ]:
def train():
  config_defaults={
      'batch_size_replay': 300,
      'memory_capacity': 500,
      'optimizer':'rmsprop',
      'epsilon': 5e-5
  }

  wandb.init(config=config_defaults,settings=wandb.Settings(start_method='fork'))
  config = wandb.config

  env = Environment()#day0 = DAY0, dayn=DAYN)

  stateCnt = env.env.observation_space.shape[0]
  actionCnt = env.env.action_space.n

  agent = Agent(stateCnt, actionCnt,memory_capacity= config.memory_capacity, batch_size=config.batch_size_replay, epsilon=config.epsilon, opt=config.optimizer)

  import time
  t0=time.time()
  for _ in range(700):
      env.run(agent)

  for day in range(DAY0,DAYN):
    wandb.gym.monitor()
    env_test.run(agent,day=day)

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="brunobo", project="DRL SARSA microgrid hyperparams")

Create sweep with ID: x5otzo5v
Sweep URL: https://wandb.ai/brunobo/DRL%20SARSA%20microgrid%20hyperparams/sweeps/x5otzo5v


In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 0rl7wdtn with config:
wandb: 	batch_size_replay: 300
wandb: 	epsilon: 1e-06
wandb: 	memory_capacity: 500
wandb: 	optimizer: rmsprop
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


100
Day: 0
Day  0
R=  -0.25513198070447757
Day: 0
Day  0
R=  -0.033969202628050636
Day: 0
Day  0
R=  -0.052598690256361866
Day: 0
Day  0
R=  -0.08181388177015333
Day: 0
Day  0
R=  -0.06476232011307403
Day: 0
Day  0
R=  -0.12285928145215193
Day: 0
Day  0
R=  -0.1184070822747299
Day: 0
Day  0
R=  -0.11381375897160827
Day: 0
Day  0
R=  -0.09694257303023383
Day: 0
Day  0
R=  -0.09592703446685742
Day: 0
Day  0
R=  -0.13971400553303504
Day: 0
Day  0
R=  -0.09372873315295795
Day: 0
Day  0
R=  -0.06224926283981303
Day: 0
Day  0
R=  -0.07710321913157259
Day: 0
Day  0
R=  -0.07140732159995361
Day: 0
Day  0
R=  -0.14108096950925553
Day: 0
Day  0
R=  -0.23485723579885107
Day: 0
Day  0
R=  -0.25182336146982337
Day: 0
Day  0
R=  -0.2612454538488296
Day: 0
Day  0
R=  -0.321679020046885
Day: 0
Day  0
R=  -0.2880482906356406
Day: 0
Day  0
R=  -0.16072081333784982
Day: 0
Day  0
R=  -0.28656597198667216
Day: 0
Day  0
R=  -0.2024078907602435
Day: 0
Day  0
R=  -0.34336218894366727
Day: 0
Day  0
R=  -0.1818

Day,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Reward,▃▁▁▄██████▇███▇█████████████████████████
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
run_count,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Day,0
GFLOPs,1e-05
Reward,0.10158
epoch,0
loss,1e-05
run_count,699


Run 0rl7wdtn errored: NameError("name 'env_test' is not defined")
wandb: ERROR Run 0rl7wdtn errored: NameError("name 'env_test' is not defined")
wandb: Agent Starting Run: i5zdk0ms with config:
wandb: 	batch_size_replay: 300
wandb: 	epsilon: 1e-05
wandb: 	memory_capacity: 475
wandb: 	optimizer: rmsprop
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


100
Day: 0
Day  0
R=  -0.2441777107318973
Day: 0
Day  0
R=  -0.19090616275863487
Day: 0
Day  0
R=  -0.259641563270291
Day: 0
Day  0
R=  -0.21889104800749726
Day: 0
Day  0
R=  -0.20429995287651662
Day: 0
Day  0
R=  -0.09008914810854546
Day: 0
Day  0
R=  -0.530093718435747
Day: 0
Day  0
R=  -0.2189876770080037
Day: 0
Day  0
R=  -0.21012088068470813
Day: 0
Day  0
R=  -0.22168764608846087
Day: 0
Day  0
R=  -0.12498472755865164
Day: 0
Day  0
R=  -0.19876421505635394
Day: 0
Day  0
R=  -0.061625633396401584
Day: 0
Day  0
R=  -0.10597293043109002
Day: 0
Day  0
R=  -0.07880911744950939
Day: 0
Day  0
R=  -0.0590221065363485
Day: 0
Day  0
R=  -0.044065999354078664
Day: 0
Day  0
R=  -0.06375932910901547
Day: 0
Day  0
R=  -0.07280093510738211
Day: 0
Day  0
R=  -0.05864689710546668
Day: 0
Day  0
R=  0.029726961721045803
Day: 0
Day  0
R=  0.03307069040498291
Day: 0
Day  0
R=  0.035219992970589586
Day: 0
Day  0
R=  0.03860626828566384
Day: 0
Day  0
R=  0.05908363099569029
Day: 0
Day  0
R=  0.082554943

...

Day,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Reward,▁▃▇█▇████████████▇██▇██████▇████████████
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
run_count,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Day,0
GFLOPs,1e-05
Reward,0.10387
epoch,0
loss,1e-05
run_count,699


Run eb58u2y3 errored: NameError("name 'env_test' is not defined")
wandb: ERROR Run eb58u2y3 errored: NameError("name 'env_test' is not defined")
wandb: Agent Starting Run: rh7rl404 with config:
wandb: 	batch_size_replay: 325
wandb: 	epsilon: 5e-06
wandb: 	memory_capacity: 500
wandb: 	optimizer: rmsprop
/usr/local/lib/python3.7/dist-packages/tornado/httputil.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class HTTPHeaders(collections.MutableMapping):


100
Day: 0
Day  0
R=  -0.20814145954030308
Day: 0
Day  0
R=  -0.06915359014150639
Day: 0
Day  0
R=  -0.1449465508448173
Day: 0
Day  0
R=  -0.16460685706612657
Day: 0
Day  0
R=  -0.2514808856798981
Day: 0
Day  0
R=  -0.2422713333762288
Day: 0
Day  0
R=  -0.24290904989477322
Day: 0
Day  0
R=  -0.2526106768169316
Day: 0
Day  0
R=  -0.24966271283369296
Day: 0
Day  0
R=  -0.28777398390319003
Day: 0
Day  0
R=  -0.2964122800313374
Day: 0
Day  0
R=  -0.231663214836511
Day: 0
Day  0
R=  -0.1705260747041841
Day: 0
Day  0
R=  -0.2578911268599694
Day: 0
Day  0
R=  -0.1803007126445116
Day: 0
Day  0
R=  -0.20553116999144483
Day: 0
Day  0
R=  -0.21600360407858826
Day: 0
Day  0
R=  -0.2052158220364484
Day: 0
Day  0
R=  -0.1944938694460671
Day: 0
Day  0
R=  -0.18109362562044723
Day: 0
Day  0
R=  -0.16971132800450128
Day: 0
Day  0
R=  -0.12417868257621861
Day: 0
Day  0
R=  -0.12171531388497789
Day: 0
Day  0
R=  -0.14557515040347277
Day: 0
Day  0
R=  -0.13983284687209024
Day: 0
Day  0
R=  -0.167299550081

Day,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Reward,▁▃▆▇████████████████████████████████████
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
run_count,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Day,0
GFLOPs,1e-05
Reward,0.10078
epoch,0
loss,1e-05
run_count,699


Run rh7rl404 errored: NameError("name 'env_test' is not defined")
wandb: ERROR Run rh7rl404 errored: NameError("name 'env_test' is not defined")
Detected 5 failed runs in a row at start, killing sweep.
wandb: ERROR Detected 5 failed runs in a row at start, killing sweep.
wandb: To change this value set WANDB_AGENT_MAX_INITIAL_FAILURES=val
